# Data Preprocessing

The following tutorial contains Python examples for data preprocessing. You should refer to the "Data" chapter of the "Introduction to Data Mining" book (slides are available at https://www-users.cs.umn.edu/~kumar001/dmbook/index.php) to understand some of the concepts introduced in this tutorial. The notebook can be downloaded from http://www.cse.msu.edu/~ptan/dmbook/tutorials/tutorial4/tutorial4.ipynb.


O pré-processamento de dados consiste em um amplo conjunto de técnicas para limpeza, seleção e transformação de dados para aprimorar a análise de mineração de dados. Leia as instruções passo a passo abaixo com cuidado. Para executar o código, clique na célula correspondente e pressione as teclas SHIFT-ENTER simultaneamente.

## 4.1 Problemas de qualidade de dados

A baixa qualidade dos dados pode ter um efeito adverso na mineração de dados. Entre os problemas comuns de qualidade de dados estão ruído, valores discrepantes, valores ausentes e dados duplicados. Esta seção apresenta exemplos de código Python para tratar alguns desses problemas de qualidade de dados. Começamos com um conjunto de dados de exemplo do repositório de aprendizado de máquina da UCI que contém informações sobre pacientes com câncer de mama. Primeiro, faremos o download do conjunto de dados usando a função Pandas read_csv() e exibiremos seus primeiros 5 pontos de dados.

**<font color="red">Code:</font>**

In [ ]:
import pandas as pd
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', header=None)
data.columns = ['Sample code', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                'Normal Nucleoli', 'Mitoses','Class']

data = data.drop(['Sample code'],axis=1)
print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))
data.head()

### Valores faltantes  (Missing Values)

Não é incomum que um objeto não tenha um ou mais valores de atributo. Em alguns casos, as informações não foram coletadas; enquanto em outros casos, alguns atributos são inaplicáveis às instâncias de dados. Esta seção apresenta exemplos das diferentes abordagens para lidar com valores ausentes.

De acordo com a descrição dos dados (https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original), os valores omissos são codificados como '?' Nos dados originais. Nossa primeria tareda é converter os valores ausentes em NaNs. Podemos então contar o número de valores ausentes em cada coluna dos dados.
**<font color="red">Code:</font>**

In [ ]:
import numpy as np

data = data.replace('?',np.NaN)

print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))

print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))

Observe que apenas a coluna 'Bare Nuclei' contém valores omissos. No exemplo a seguir, os valores ausentes na coluna "Bare Nuclei" são substituídos pelo valor mediano dessa coluna. Os valores antes e depois da substituição são mostrados para um subconjunto dos pontos de dados.

**<font color="red">Code:</font>**

In [ ]:
data2 = data['Bare Nuclei']

print('Before replacing missing values:')
print(data2[20:25])
data2 = data2.fillna(data2.median())

print('\nAfter replacing missing values:')
print(data2[20:25])

Em vez de substituir os valores ausentes, outra abordagem comum é descartar os pontos de dados que contêm valores ausentes. Isso pode ser feito facilmente aplicando a função dropna () ao quadro de dados.
**<font color="red">Code:</font>**

In [ ]:
print('Number of rows in original data = %d' % (data.shape[0]))

data2 = data.dropna()
print('Number of rows after discarding missing values = %d' % (data2.shape[0]))

### Outliers (Ruídos)

Outliers are data instances with characteristics that are considerably different from the rest of the dataset. In the example code below, we will draw a boxplot to identify the columns in the table that contain outliers. Note that the values in all columns (except for 'Bare Nuclei') are originally stored as 'int64' whereas the values in the 'Bare Nuclei' column are stored as string objects (since the column initially contains strings such as '?' for representing missing values). Thus, we must  convert the column into numeric values first before creating the boxplot. Otherwise, the column will not be displayed when drawing the boxplot.

Os outliers são instâncias de dados com características consideravelmente diferentes do restante do conjunto de dados. No código de exemplo abaixo, desenharemos um boxplot para identificar as colunas na tabela que contêm outliers. Observe que os valores em todas as colunas (exceto 'Bare Nuclei') são originalmente armazenados como 'int64', enquanto os valores na coluna 'Bare Nuclei' são armazenados como objetos de string (já que a coluna contém strings como '?' representando valores ausentes). Assim, devemos converter a coluna em valores numéricos antes de criar o boxplot. Caso contrário, a coluna não será exibida ao desenhar o boxplot.

**<font color="red">Code:</font>**

In [ ]:
%matplotlib inline

data2 = data.drop(['Class'],axis=1)
data2['Bare Nuclei'] = pd.to_numeric(data2['Bare Nuclei'])
data2.boxplot(figsize=(20,3))

Os boxplots sugerem que apenas 5 das colunas (Marginal Adhesion, Single Epithetial Cell Size, Bland Cromatin, Normal Nucleoli, and Mitoses) contêm valores anormalmente altos. Para descartar os outliers, podemos calcular a pontuação Z para cada atributo e remover as instâncias que contêm atributos com pontuação Z anormalmente alta ou baixa (por exemplo, se Z > 3 ou Z <= -3).

**<font color="red">Code:</font>**

O código a seguir mostra os resultados da padronização das colunas dos dados. Observe que os valores omissos (NaN) não são afetados pelo processo de padronização.

In [ ]:
Z = (data2-data2.mean())/data2.std()
Z[20:25]

**<font color="red">Code:</font>**

O código a seguir mostra os resultados de descartar colunas com Z> 3 ou Z <= -3.

In [ ]:
print('Number of rows before discarding outliers = %d' % (Z.shape[0]))

Z2 = Z.loc[((Z > -3).sum(axis=1)==9) & ((Z <= 3).sum(axis=1)==9),:]
print('Number of rows after discarding missing values = %d' % (Z2.shape[0]))

### Dados duplicados

Some datasets, especially those obtained by merging multiple data sources, may contain duplicates or near duplicate instances. The term deduplication is often used to refer to the process of dealing with duplicate data issues. 

Alguns conjuntos de dados, especialmente aqueles obtidos pela fusão de várias fontes de dados, podem conter duplicações ou instâncias quase duplicadas. O termo deduplicação é freqüentemente usado para se referir ao processo de lidar com problemas de dados duplicados.

**<font color="red">Code:</font>**

No exemplo a seguir, primeiro verificamos instâncias duplicadas no conjunto de dados de câncer de mama.

In [ ]:
dups = data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))
data.loc[[11,28]]

In [ ]:
data.loc[dups][:5]

A função duplicated () retornará uma matriz booleana que indica se cada linha é uma duplicata de uma linha anterior na tabela. Os resultados sugerem que há 236 linhas duplicadas no conjunto de dados do câncer de mama. Por exemplo, a instância com índice de linha 11 possui valores de atributo idênticos à instância com índice de linha 28. Embora essas linhas duplicadas possam corresponder a amostras para indivíduos diferentes, neste exemplo hipotético, supomos que as duplicatas são amostras obtidas do mesmo indivíduo e ilustre abaixo como remover as linhas duplicadas.

**<font color="red">Code:</font>**

In [ ]:
print('Number of rows before discarding duplicates = %d' % (data.shape[0]))
data2 = data.drop_duplicates()
print('Number of rows after discarding duplicates = %d' % (data2.shape[0]))

## Agregação

A agregação de dados é uma tarefa de pré-processamento em que os valores de dois ou mais objetos são combinados em um único objeto. A motivação para agregação inclui (1) reduzir o tamanho dos dados a serem processados, (2) alterar a granularidade da análise (de escala fina para escala mais grosseira) e (3) melhorar a estabilidade dos dados.

No exemplo abaixo, usaremos os dados da série temporal de precipitação diária para uma estação meteorológica localizada no Detroit Metro Airport. Os dados brutos foram obtidos do site Climate Data Online (https://www.ncdc.noaa.gov/cdo-web/). A série temporal de precipitação diária será comparada com os valores mensais.

**<font color="red">Code:</font>**

O código abaixo carrega os dados da série temporal de precipitação e desenha um gráfico de linha das suas séries temporais diárias.

In [ ]:
daily = pd.read_csv('DTW_prec.csv', header='infer')
daily.index = pd.to_datetime(daily['DATE'])
daily.head()

In [ ]:
daily = daily['PRCP']
ax = daily.plot(kind='line',figsize=(15,3))
ax.set_title('Daily Precipitation (variance = %.4f)' % (daily.var()))

Observe que a série temporal diária parece ser bastante caótica e varia significativamente de um passo para outro. As séries temporais podem ser agrupadas e agregadas por mês para obter os valores totais mensais de precipitação. A série temporal resultante parece variar mais suavemente em comparação com a série temporal diária.

**<font color="red">Code:</font>**

In [ ]:
monthly = daily.groupby(pd.Grouper(freq='M')).sum()


In [ ]:
monthly[:10]

In [ ]:
ax = monthly.plot(kind='line',figsize=(15,3))
ax.set_title('Monthly Precipitation (variance = %.4f)' % (monthly.var()))

No exemplo abaixo, as séries temporais diárias de precipitação são agrupadas e agregadas por ano para obter os valores de precipitação anuais.

**<font color="red">Code:</font>**

In [ ]:
annual = daily.groupby(pd.Grouper(freq='Y')).sum()
annual[:5]

In [ ]:
ax = annual.plot(kind='line',figsize=(15,3))
ax.set_title('Annual Precipitation (variance = %.4f)' % (annual.var()))

## Sampling

Sampling is an approach commonly used to facilitate (1) data reduction for exploratory data analysis and scaling up algorithms to big data applications and (2) quantifying uncertainties due to varying data distributions. There are various methods available for data sampling, such as sampling without replacement, where each selected instance is removed from the dataset, and sampling with replacement, where each selected instance is not removed, thus allowing it to be selected more than once in the sample.

Amostragem é uma abordagem comumente usada para facilitar (1) a redução de dados para análise exploratória de dados e ampliação de algoritmos para aplicações de big data e (2) quantificação de incertezas devido a distribuições de dados variáveis. Existem vários métodos disponíveis para amostragem de dados, como amostragem sem substituição, onde cada instância selecionada é removida do conjunto de dados e amostragem com substituição, onde cada instância selecionada não é removida, permitindo que ela seja selecionada mais de uma vez na amostra .

No exemplo abaixo, aplicaremos amostragem com substituição e sem substituição ao conjunto de dados de câncer de mama obtido no repositório de aprendizado de máquina da UCI.

**<font color="red">Code:</font>**

Inicialmente exibimos os cinco primeiros registros da tabela.

In [ ]:
data.head()

No código a seguir, uma amostra de tamanho 3 é selecionada aleatoriamente (sem substituição) dos dados originais.

**<font color="red">Code:</font>**

In [ ]:
sample = data.sample(n=3)
sample

No próximo exemplo, selecionamos aleatoriamente 1% dos dados (sem substituição) e exibimos as amostras selecionadas. O argumento random_state da função especifica o valor inicial do gerador de números aleatórios.

**<font color="red">Code:</font>**

In [ ]:
sample = data.sample(frac=0.01, random_state=1)
sample

Por fim, realizamos uma amostragem com substituição para criar uma amostra cujo tamanho é igual a 1% de todos os dados. Você deve poder observar instâncias duplicadas na amostra ao aumentar o tamanho da amostra.

**<font color="red">Code:</font>**

In [ ]:
sample = data.sample(frac=0.01, replace=True, random_state=1)
sample

## Discretização

Discretização é uma etapa de pré-processamento de dados que é frequentemente usada para transformar um atributo de valor contínuo em um atributo categórico. O exemplo abaixo ilustra dois métodos de discretização simples, mas amplamente utilizados (largura igual e freuquência igual), aplicados ao atributo 'Clump Thickness' do conjunto de dados de câncer de mama.

Primeiro, plotamos um histograma que mostra a distribuição dos valores dos atributos. A função value_counts() também pode ser aplicada para contar a frequência de cada valor de atributo.

**<font color="red">Code:</font>**

In [ ]:
data['Clump Thickness'].hist(bins=10)
data['Clump Thickness'].value_counts(sort=False)

Para o método de largura igual, podemos aplicar a função cut() para discretizar o atributo em 4 bins de larguras de intervalo semelhantes. A função value_counts() pode ser usada para determinar o número de instâncias em cada bin.

**<font color="red">Code:</font>**

In [ ]:
bins = pd.cut(data['Clump Thickness'],4)
bins.value_counts(sort=False)

Para o método de frequência igual, a função qcut() pode ser usada para particionar os valores em 4 compartimentos, de forma que cada bin tenha quase o mesmo número de instâncias.

**<font color="red">Code:</font>**

In [ ]:
bins = pd.qcut(data['Clump Thickness'],4)
bins.value_counts(sort=False)


**<font color='blue'>References:</font>**

1. Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science. 

2. Mangasarian, O.L. and Wolberg, W. H. (1990). "Cancer diagnosis via linear programming", SIAM News, Volume 23, Number 5, pp 1 & 18.

3. Wolberg, W.H. and Mangasarian, O.L. (1990). "Multisurface method of pattern separation for medical diagnosis applied to breast cytology", Proceedings of the National Academy of Sciences, U.S.A., Volume 87, pp 9193-9196. 

4. Climate Data Online [https://www.ncdc.noaa.gov/cdo-web/]. 